In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, TimeDistributed, RepeatVector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# File path
base_path = r"C:\Users\Admin\Downloads\processed_subjects - Copy\processed_subjects - Copy\final_processed"

# Subjects
autistic_subjects = [f"{29864 + i}" for i in range(27) if 29864 + i not in [29864, 29887]]
non_autistic_subjects = [f"{29864 + i}" for i in range(27, 54) if 29864 + i not in [29864, 29887]]

# Labels: 1 for autism, 0 for non-autism
subjects = autistic_subjects + non_autistic_subjects
labels = [1] * len(autistic_subjects) + [0] * len(non_autistic_subjects)

print(subjects)


['29865', '29866', '29867', '29868', '29869', '29870', '29871', '29872', '29873', '29874', '29875', '29876', '29877', '29878', '29879', '29880', '29881', '29882', '29883', '29884', '29885', '29886', '29888', '29889', '29890', '29891', '29892', '29893', '29894', '29895', '29896', '29897', '29898', '29899', '29900', '29901', '29902', '29903', '29904', '29905', '29906', '29907', '29908', '29909', '29910', '29911', '29912', '29913', '29914', '29915', '29916', '29917']


In [11]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, TimeDistributed, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau

# File path
base_path = r"C:\Users\Admin\Downloads\processed_subjects - Copy\processed_subjects - Copy\final_processed"

# Subjects
autistic_subjects = [f"{29864 + i}" for i in range(27) if 29864 + i not in [29864, 29887]]
non_autistic_subjects = [f"{29864 + i}" for i in range(27, 54) if 29864 + i not in [29864, 29887]]

# Labels: 1 for autism, 0 for non-autism
subjects = autistic_subjects + non_autistic_subjects
labels = [1] * len(autistic_subjects) + [0] * len(non_autistic_subjects)

# Load data
data = []
for subject in subjects:
    file_path = os.path.join(base_path, f"{subject}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, header=None)
        data.append(df.values)

data = np.array(data)  # Shape: (subjects, time_points, features)
labels = np.array(labels)

# Normalize data
scaler = MinMaxScaler()
data_normalized = np.array(data)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

# Autoencoder for feature reduction
input_dim = data_normalized.shape[2]
time_steps = data_normalized.shape[1]

input_layer = Input(shape=(time_steps, input_dim))
encoded = TimeDistributed(Dense(500, activation="relu"))(input_layer)
encoded = TimeDistributed(Dense(300, activation="relu"))(encoded)
encoded = TimeDistributed(Dense(150, activation="relu"))(encoded)
bottleneck = TimeDistributed(Dense(75, activation="relu"))(encoded)

decoded = TimeDistributed(Dense(150, activation="relu"))(bottleneck)
decoded = TimeDistributed(Dense(300, activation="relu"))(decoded)
decoded = TimeDistributed(Dense(500, activation="relu"))(decoded)
decoded = TimeDistributed(Dense(input_dim, activation="sigmoid"))(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_train, X_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)

# Reduced features
encoder = Model(input_layer, bottleneck)
X_train_reduced = encoder.predict(X_train)
X_test_reduced = encoder.predict(X_test)

# Reshape for LSTM
X_train_reduced = X_train_reduced.reshape(X_train_reduced.shape[0], time_steps, -1)
X_test_reduced = X_test_reduced.reshape(X_test_reduced.shape[0], time_steps, -1)

# LSTM model for classification
model = Sequential([
    Bidirectional(LSTM(128, input_shape=(time_steps, X_train_reduced.shape[2]), return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    LSTM(32, return_sequences=False),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train LSTM
model.fit(X_train_reduced, y_train, epochs=50, batch_size=8, validation_split=0.2, callbacks=[lr_scheduler], verbose=1)

# Evaluate
loss, accuracy = model.evaluate(X_test_reduced, y_test, verbose=1)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - loss: 2065.0664 - val_loss: 2063.6873
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 2063.4946 - val_loss: 2062.8491
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - loss: 2062.7856 - val_loss: 2062.5437
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - loss: 2062.5156 - val_loss: 2062.3979
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - loss: 2062.4028 - val_loss: 2062.3474
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - loss: 2062.3643 - val_loss: 2062.3071
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - loss: 2062.3240 - val_loss: 2062.2800
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - loss: 2062.2847 - val_loss: 2062.2275
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - loss: 2062.2314 - val_loss: 2062.1738
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 2062.1748 - val_loss: 2062.1101
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - loss: 2062.1167 - val_loss: 2062.0837
Epoch 12/5

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 230ms/step - accuracy: 0.4958 - loss: 0.7501 - val_accuracy: 0.4444 - val_loss: 0.7157 - learning_rate: 0.0010
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6333 - loss: 0.7048 - val_accuracy: 0.4444 - val_loss: 0.7603 - learning_rate: 0.0010
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5542 - loss: 0.6694 - val_accuracy: 0.4444 - val_loss: 0.7653 - learning_rate: 0.0010
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5708 - loss: 0.6995 - val_accuracy: 0.4444 - val_loss: 0.7276 - learning_rate: 0.0010
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6750 - loss: 0.6582 - val_accuracy: 0.3333 - val_loss: 0.7320 - learning_rate: 0.0010
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4375 - loss: 0.7050 - val_accuracy: 0.4444 - val_loss: 0.7732 - learning_rate: 0.0010
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5875 - loss: 0.6423 - val_accuracy: 0.3333 - val_loss

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Predict on the test set
y_pred_prob = model.predict(X_test_reduced)  # Predicted probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Non-Autistic", "Autistic"]))

# Confusion Matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
Classification Report:
              precision    recall  f1-score   support

Non-Autistic       0.80      0.80      0.80         5
    Autistic       0.83      0.83      0.83         6

    accuracy                           0.82        11
   macro avg       0.82      0.82      0.82        11
weighted avg       0.82      0.82      0.82        11

Confusion Matrix:
[[4 1]
 [1 5]]
ROC-AUC Score: 0.9000
